<a href="https://colab.research.google.com/github/Nongkynrih/SIGN/blob/main/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"johnanongkynrih","key":"528d37840d2297b10b195b57ec55167b"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list

ref                                                               title                                         size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------------------  -------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
asinow/car-price-dataset                                          Car Price Dataset                            135KB  2025-01-26 19:53:28          11299        157  1.0              
adilshamim8/education-and-career-success                          Education & Career Success.                  118KB  2025-02-03 05:24:20           2472         45  1.0              
anandshaw2001/netflix-movies-and-tv-shows                         Netflix Movies and TV Shows                    1MB  2025-01-03 10:33:01          19486        480  1.0              
hubertsidorowicz/football-players-stats-2024-2025                 Football Players St

In [ ]:
!kaggle datasets list -s WLASL

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.11/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [ ]:
!pip install kagglehub==0.1.9

  Attempting uninstall: kagglehub
    Found existing installation: kagglehub 0.3.7
    Uninstalling kagglehub-0.3.7:
      Successfully uninstalled kagglehub-0.3.7


In [ ]:
import kagglehub

In [ ]:
!kaggle datasets download waseemnagahhenes/sign-language-dataset-wlasl-videos

Dataset URL: https://www.kaggle.com/datasets/waseemnagahhenes/sign-language-dataset-wlasl-videos
License(s): unknown
100% 4.88G/4.90G [01:02<00:00, 150MB/s]
100% 4.90G/4.90G [01:02<00:00, 83.6MB/s]


In [ ]:
!unzip sign-language-dataset-wlasl-videos.zip

Streaming output truncated to the last 5000 lines.
  inflating: dataset/SL/nun/39080.mp4  
  inflating: dataset/SL/nun/39082.mp4  
  inflating: dataset/SL/nurse/39088.mp4  
  inflating: dataset/SL/nurse/39089.mp4  
  inflating: dataset/SL/nurse/39090.mp4  
  inflating: dataset/SL/nurse/39091.mp4  
  inflating: dataset/SL/nurse/39092.mp4  
  inflating: dataset/SL/nurse/39093.mp4  
  inflating: dataset/SL/nurse/39099.mp4  
  inflating: dataset/SL/nut/39115.mp4  
  inflating: dataset/SL/nut/39116.mp4  
  inflating: dataset/SL/nut/39118.mp4  
  inflating: dataset/SL/nut/39120.mp4  
  inflating: dataset/SL/nut/66220.mp4  
  inflating: dataset/SL/o/39553.mp4  
  inflating: dataset/SL/o/39554.mp4  
  inflating: dataset/SL/o/39555.mp4  
  inflating: dataset/SL/o/39557.mp4  
  inflating: dataset/SL/o/66053.mp4  
  inflating: dataset/SL/objective/39198.mp4  
  inflating: dataset/SL/objective/39199.mp4  
  inflating: dataset/SL/objective/39200.mp4  
  inflating: dataset/SL/objective/39201.mp4  
 

In [ ]:
!ls

dataset  kaggle.json  processed_videos	sample_data  sign-language-dataset-wlasl-videos.zip


In [ ]:
!ls dataset/SL

 a		     congress	      government	 olympics	   slave
 abdomen	     connect	      grab		 on		   sleep
 able		     conquer	      graduate		 once		   sleepy
 about		     consider	      graduation	 one		   slice
 above		     constitution     grammar		 onion		   slip
 accent		     construct	      grandfather	 only		   slow
 accept		     consume	      grandma		 open		   small
 accident	     contact	      grandmother	 operate	   smart
 accomplish	     contest	      grapes		 opinion	   smell
 accountant	     continue	      grass		 opposite	   smile
 across		     contract	      grateful		 or		   smoking
 act		     contribute       gray		 oral		   smooth
 action		     control	      great		 orange		   snack
 active		     conversation     greece		 order		   snake
 activity	     convert	      green		 organize	   sneeze
 actor		     convince	      grey		 other		   snob
 adapt		     cook	      group		 our		   snow
 add		     cookie	      grow		 out		   snowman
 address	     cool	     'grow up'		 out

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Define image properties
IMG_SIZE = 224  # Resize all images to 224x224
DATASET_PATH = "dataset/SL"  # Update with your dataset path
CLASSES = sorted(os.listdir(DATASET_PATH))  # Get class labels from folder names
FRAME_SKIP = 5  # Extract every 5th frame to reduce redundancy

# Create label mapping
label_map = {label: idx for idx, label in enumerate(CLASSES)}

def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % FRAME_SKIP == 0:  # Extract every FRAME_SKIP frame
            frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))  # Resize
            frame = frame / 255.0  # Normalize pixel values
            frames.append(frame)
        frame_count += 1
    cap.release()
    return frames

def data_generator(batch_size=32):
    while True:
        X, y = [], []
        for label in CLASSES:
            class_dir = os.path.join(DATASET_PATH, label)
            for video_name in os.listdir(class_dir):
                video_path = os.path.join(class_dir, video_name)
                frames = extract_frames(video_path)
                X.extend(frames)
                y.extend([label_map[label]] * len(frames))
                if len(X) >= batch_size:
                    X_array = np.array(X[:batch_size], dtype=np.float32)
                    y_array = np.array(y[:batch_size], dtype=np.int32)
                    y_array = to_categorical(y_array, num_classes=len(CLASSES))
                    yield X_array, y_array
                    X, y = [], []  # Reset after yielding

# Example: Using generator to fetch one batch
data_gen = data_generator(batch_size=32)
X_batch, y_batch = next(data_gen)
print(f"✅ Preprocessing complete! Batch shape: {X_batch.shape}, Labels shape: {y_batch.shape}")


✅ Preprocessing complete! Batch shape: (32, 224, 224, 3), Labels shape: (32, 2000)


In [ ]:
import cv2
import os
import numpy as np

# Define video properties
FRAME_COUNT = 30  # Number of frames to sample per video
IMG_SIZE = 224  # Resize frames
VIDEO_PATH = "dataset/SL"  # Path to the dataset
PROCESSED_DATA_PATH = "processed_videos"  # Output path

# Create output directory
os.makedirs(PROCESSED_DATA_PATH, exist_ok=True)

def extract_frames(video_path, frame_count=FRAME_COUNT):
    """Extract frames from a video file."""
    cap = cv2.VideoCapture(video_path)
    frames = []

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames < frame_count:
        frame_count = total_frames  # Adjust frame count if video has fewer frames

    frame_indices = np.linspace(0, total_frames - 1, frame_count, dtype=int)  # Sample evenly

    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        if i in frame_indices:
            frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))  # Resize frame
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB
            frame = frame / 255.0  # Normalize
            frames.append(frame)

    cap.release()
    return np.array(frames, dtype=np.float32)

# Process all videos
for class_label in os.listdir(VIDEO_PATH):
    class_dir = os.path.join(VIDEO_PATH, class_label)
    if not os.path.isdir(class_dir):
        continue  # Skip non-directory files

    save_dir = os.path.join(PROCESSED_DATA_PATH, class_label)
    os.makedirs(save_dir, exist_ok=True)

    for video_file in os.listdir(class_dir):
        if not video_file.endswith(('.mp4', '.avi', '.mov')):
            continue  # Skip non-video files

        video_path = os.path.join(class_dir, video_file)
        frames = extract_frames(video_path)

        if frames.shape[0] > 0:  # Ensure frames were extracted
            save_path = os.path.join(save_dir, os.path.splitext(video_file)[0] + ".npy")
            try:
                np.save(save_path, frames)  # Save as NumPy array
            except OSError:
                print(f"⚠️ Not enough space to save {save_path}, skipping.")

print("✅ Video preprocessing complete!")


⚠️ Not enough space to save processed_videos/home/69366.npy, skipping.


OSError: [Errno 28] No space left on device: 'processed_videos/avoid'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')